In [2]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#import psycopg2
#from config import db_password
import time
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf

In [3]:
dpo_table = pd.read_csv('dpo_table.csv')
#pkg_dim_table = pd.read_csv('pkg_dims_table.csv')
#region = pd.read_csv('region_table.csv')

In [9]:
dpo_table

,actdely_dt,omstracknum,ordnum,origin_zip,dest_zip,shipnode,channel,carrier_scac,reqship_dt,sysship_dttm,...,node_on_time_ship,ots_rsd,post_release_inf,node_mnr,carrier_not_meet_tnt,carrier_defect,origin_defect,system_defect,uncontrollable_defect,dpo_delivered_late
0,2019-02-03,1Z2E24F1YW37836926,1022331943153,17408,60126,9475,FC 9475,UPSN,2019-01-30,2019-01-29 20:25,...,NaN,0.0,0.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N
1,2019-02-03,420100169261290982827026490071,9250073345592,7047,10016,STR1865,STR,USPS,2019-02-01,2019-02-01 15:51,...,Node: On Time Ship,0.0,0.0,NaN,NaN,NaN,origin,NaN,NaN,N
2,2019-02-03,420100259261290982827019489617,3550521247344,7047,10025,STR1865,STR,USPS,2019-02-02,2019-02-02 12:08,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,N
3,2019-02-03,420100269261290982827013614626,9050522892943,7047,10026,STR1865,STR,USPS,2019-02-02,2019-02-02 13:21,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,N
4,2019-02-03,420100289261290203606120962697,1050507158738,10451,10028,STR2475,STR,USPS,2019-02-01,2019-01-31 17:22,...,NaN,0.0,0.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N
5,2019-02-03,420100319261290203606117430789,9050500336068,10451,10031,STR2475,STR,USPS,2019-02-01,2019-01-31 21:07,...,NaN,1.0,1.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N
6,2019-02-03,420100659261290203606112330947,1050520783130,10451,10065,STR2475,STR,USPS,2019-02-02,2019-02-01 10:59,...,NaN,0.0,0.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N
7,2019-02-03,420105389261290982830516198944,9250521424633,10977,10538,STR1808,STR,USPS,2019-02-02,2019-02-02 14:51,...,Node: On Time Ship,0.0,0.0,NaN,NaN,NaN,origin,NaN,NaN,N
8,2019-02-03,420191439274890982839321335236,9050514159824,19150,19143,STR2527,STR,USPS,2019-02-01,2019-02-01 16:10,...,Node: On Time Ship,0.0,0.0,NaN,NaN,NaN,origin,NaN,NaN,N
9,2019-02-03,420208329261290203606325572608,9050150682786,20878,20832,STR1193,STR,USPS,2019-02-01,2019-02-01 15:09,...,Node: On Time Ship,0.0,0.0,NaN,NaN,NaN,origin,NaN,NaN,N


In [29]:
for column in dpo_table.columns:
    print(f"Column {column} has {dpo_table[column].isnull().sum()} null values")

Column actdely_dt has 0 null values
Column omstracknum has 0 null values
Column ordnum has 0 null values
Column origin_zip has 0 null values
Column dest_zip has 0 null values
Column shipnode has 0 null values
Column channel has 0 null values
Column carrier_scac has 0 null values
Column reqship_dt has 0 null values
Column sysship_dttm has 0 null values
Column sysship_dt has 0 null values
Column estdely_dt has 0 null values
Column actdely_month has 0 null values
Column estdely_week has 0 null values
Column delivered_late has 0 null values
Column xcpnscan1_desc has 0 null values
Column xcpnscan2_desc has 0 null values
Column xcpnscan3_desc has 0 null values
Column xcpnscan4_desc has 0 null values
Column planned_transit has 0 null values
Column carrier_pub_tnt has 0 null values
Column units has 0 null values
Column transit_override has 0 null values
Column carrier_xcp has 0 null values
Column guest_xcp has 0 null values
Column holiday_xcp has 0 null values
Column weather_xcp has 0 null val

In [12]:
dpo_table = dpo_table.drop(['system_defect','carrier_defect'], axis=1)

In [23]:

dpo_table['actdely_dt'] = pd.to_datetime(dpo_table['actdely_dt'])

In [24]:
y = dpo_table['dpo_delivered_late']
X = dpo_table.drop(columns = 'dpo_delivered_late')

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [26]:
X_train.shape

(37, 37)

In [27]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [28]:
classifier.fit(X_train, y_train)

TypeError: The DType <class 'numpy.dtype[float64]'> could not be promoted by <class 'numpy.dtype[datetime64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[object_]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dty